In [ ]:
from matplotlib.pyplot import *
from numpy import *
import numpy.random as npr
rcParams['text.usetex'] = True

In [ ]:
from optics_design_workbench import distributions
from optics_design_workbench.distributions import *

# test scalar random number generator

In [ ]:
for disableAnalytical in (True, False):
  figure(figsize=(3,3))

  for expect, expr, domain in [
    (lambda x: x**2, 'x**2', (0, 10)),
    (lambda x: 0.5*x+cos(x), 'x/2+cos(x)', (0, 4*pi)),
    (lambda x: exp(-(x-3)**2), 'exp(-(x-3)**2)', (1, 5)),
  ]:
    x = distributions.ScalarRandomVariable(expr, variableDomain=domain)
    x.compile(disableAnalytical=disableAnalytical)
    print(f'expr {expr} mode: {x.mode()}')
    H, bins = histogram(x.draw(1e6), bins=50)
    bins = (bins[1:]+bins[:-1])/2
    Hexpect = expect(bins)
    Hexpect /= Hexpect.max()
    H = H/H.max()
    plot(bins, H, 'x')
    plot(bins, Hexpect)
    err = sqrt(mean( ((Hexpect-H)/H.max())**2 ))
    print(f'{err=}')
    assert err < 3e-2

# test vector random number generator

In [ ]:
import scipy
scipy.special.erfinv(0.99999)

In [ ]:
x = distributions.VectorRandomVariable('exp(-theta**2)', variableDomains=dict(theta=(0,pi), phi=(0,2*pi)))
x.compile()
x.showExpressions()
x.draw()

In [ ]:
for disableAnalytical in (True, False):
  for expect, expr, order, domain in [
    (lambda x, y: exp(-(x**2 + y**2)), 'exp(-(x**2 + y**2))', ['x', 'y'], dict(x=(-1,2), y=(-3,2))),
    (lambda x, y: exp(-x**2), 'exp(-theta**2)', ['theta', 'phi'], dict(theta=(0,pi), phi=(0,2*pi))),
    (lambda x, y: exp(-x**2/(1+y/3)**2), 'exp(-theta**2/(1+phi/3)**2)', ['theta', 'phi'], dict(theta=(0,pi), phi=(0,2*pi))),
  ]:
    x = distributions.VectorRandomVariable(expr, variableDomains=domain, variableOrder=order)
    x.compile(disableAnalytical=disableAnalytical)
    print(f'expr {expr} mode: {x.mode()}')
    #x.showExpressions()

    H, binsX, binsY = histogram2d(*x.draw(1e6), bins=(50, 55))
    figure(figsize=(3,3))
    pcolormesh(binsX, binsY, H.T)
    xlabel(order[0])
    ylabel(order[1])

    X = (binsX[1:]+binsX[:-1])/2
    Y = (binsY[1:]+binsY[:-1])/2
    Hexpect = expect(*meshgrid(X, Y))
    figure(figsize=(3,3))
    pcolormesh(binsX, binsY, Hexpect)
    xlabel(order[0])
    ylabel(order[1])

    err = sqrt(mean( ((Hexpect/Hexpect.max()-H.T/H.max()))**2 ))
    print(err)
    show()
    assert err < 0.1

# test inner lambdas and their symmetry

In [ ]:
x = distributions.VectorRandomVariable(
      '.25-(x-.5)**2 + .02*y', 
      variableOrder=['y', 'x'], 
      variableDomains=dict(x=(0,1), y=(0,3)),
      numericalResolutions=dict(x=7, y=9))
x.compile(disableAnalytical=True)
x._variables

In [ ]:
x.draw(N=5)

In [ ]:
H, binsX, binsY = histogram2d(*x.draw(1e5), bins=(55, 45))
pcolormesh(binsX, binsY, H.T)

In [ ]:
# make sure desired variable order was respected
assert [str(v) for v in x._variables] == ['y', 'x']

In [ ]:
# ensure symmetry of transfer function for y variable
X = linspace(0, 1, 100)
Y = x._transformLambdas[1][0](X)
plot(X, Y)
plot(1-X, max(Y)-Y)

err = sqrt(mean( (Y - (max(Y)-Y[::-1]))**2 ))
err

In [ ]:
assert err < 1e-5

In [ ]:
# x variable function is not symmetric
X = linspace(0, 1, 100)
plot(X, x._transformLambdas[0][0](X, 1 + 0*X))